In [1]:
# Load data from Excel file
def GetData(WordFile, WordWorksheet, GridFile, GridWorksheet):
    Rank = LoadFromExcel(WordFile, WordWorksheet, 'rank')
    Frequency = LoadFromExcel(WordFile, WordWorksheet, 'frequency')
    Word = LoadFromExcel(WordFile, WordWorksheet, 'word')
    Rank.columns = ['Candidate']
    Frequency.columns = ['Candidate']
    Word.columns = ['Candidate']
    
    GridWords = LoadFromExcel(GridFile, GridWorksheet, 'NumWords')
    AcrossRef = LoadFromExcel(GridFile, GridWorksheet, 'AcrossRef')
    AcrossPos = LoadFromExcel(GridFile, GridWorksheet, 'AcrossPos')
    DownRef = LoadFromExcel(GridFile, GridWorksheet, 'DownRef')
    DownPos = LoadFromExcel(GridFile, GridWorksheet, 'DownPos')
    
    return Rank, Frequency, Word, GridWords, AcrossRef, AcrossPos, DownRef, DownPos

In [2]:
# Define model data, assigning all data to the Model
def DefineModelData(Model, Rank, Frequency, Word, GridWords, AcrossRef, AcrossPos, DownRef, DownPos):
    Size = SampleSize
    if Size == 0:
        Size = len(Rank)
    else:
        Size = min(SampleSize, len(Rank))
    print(f'Lexicon size: {Size:,.0f}')
    
    Model.Candidate = pyo.Set(initialize = range(0, Size))   # Set of candidate words 
    Model.Ascii = pyo.Set(initialize = range(0, MaxWordLength))   # Set of ASCII character codes in a word
    Model.Rank = pyo.Param(Model.Candidate, within = pyo.NonNegativeIntegers, mutable = True)   # Lexicon rank
    Model.Frequency = pyo.Param(Model.Candidate, within = pyo.NonNegativeReals, mutable = True)   # Lexicon frequency
    Model.Length = pyo.Param(Model.Candidate, within = pyo.NonNegativeIntegers, mutable = True)   # Number of characters in each candidate word
    Model.Word = pyo.Param(Model.Candidate, Model.Ascii, within = pyo.NonNegativeIntegers, mutable = True)   # Each candidate word, split into ASCII codes

    List = [i for i in range(0, len(Rank))]   # Row numbers for whole lexicon
    if Size == len(Rank):
        Sample = List   # Use whole lexicon
    else:
        Sample = rnd.sample(List, Size)   # Use sample of lexicon
    CandidateNum = 0
    for c in range(0, len(Rank)):   # Populate data for selected sample words
        if c in Sample:
            Model.Rank[CandidateNum] = Rank['Candidate'][c]
            Model.Frequency[CandidateNum] = Frequency['Candidate'][c]
            Model.Length[CandidateNum] = len(Word['Candidate'][c])
            LettersAscii = list(bytes(Word['Candidate'][c], 'ascii'))   # ASCII codes for word's characters
            for a in Model.Ascii:   # list of ASCII codes for each letter of a word, padded with zeroes beyond the word's length
                if a >= len(LettersAscii):
                    Model.Word[CandidateNum, a] = 0
                else:
                    Model.Word[CandidateNum, a] = LettersAscii[a]
            CandidateNum += 1
                
    Grid_rows, Grid_cols = np.shape(AcrossRef)   # Note: Model defined as h, w while data accessed via w, h
        
    Model.GridWords = pyo.Set(initialize = range(0, GridWords.iloc[0][0]))   # Set for number of words in the grid
    Model.GridWidth = pyo.Set(initialize = range(0, Grid_cols))   # Set width of the grid
    Model.GridHeight = pyo.Set(initialize = range(0, Grid_rows))   # Set for the height of the grid
    Model.AcrossRef = pyo.Param(Model.GridHeight, Model.GridWidth, within = pyo.NonNegativeIntegers, mutable = True)
    Model.AcrossPos = pyo.Param(Model.GridHeight, Model.GridWidth, within = pyo.NonNegativeIntegers, mutable = True)
    Model.DownRef = pyo.Param(Model.GridHeight, Model.GridWidth, within = pyo.NonNegativeIntegers, mutable = True)
    Model.DownPos = pyo.Param(Model.GridHeight, Model.GridWidth, within = pyo.NonNegativeIntegers, mutable = True)
    Model.GridLengths = pyo.Param(Model.GridWords, within = pyo.NonNegativeIntegers, mutable = True, initialize = 0)

    for w in Model.GridWidth:
        for h in Model.GridHeight:
            Model.AcrossRef[h, w] = AcrossRef[w][h]   # Populate grid encoding
            Model.AcrossPos[h, w] = AcrossPos[w][h]
            Model.DownRef[h, w] = DownRef[w][h]
            Model.DownPos[h, w] = DownPos[w][h]
            if AcrossRef[w][h] >= 1:   # Get length of "across" words by looking at maximum position of each word
                Model.GridLengths[AcrossRef[w][h] - 1] = max(pyo.value(Model.GridLengths[AcrossRef[w][h] - 1]), AcrossPos[w][h])
            if DownRef[w][h] >= 1:   # Get length of "down" words by looking at maximum position of each word
                Model.GridLengths[DownRef[w][h] - 1] = max(pyo.value(Model.GridLengths[DownRef[w][h] - 1]), DownPos[w][h])